In [1]:
import re
import datetime
import polars as pl

In [2]:
now = datetime.datetime.now()
print(now)

2024-11-03 03:58:46.936697


In [3]:
# set the threshold
threshold = 5

In [4]:
# polars method: https://docs.pola.rs/api/python/stable/reference/expressions/api/polars.Expr.str.replace_all.html#polars-expr-str-replace-all

metadata = pl.read_csv(
    "../Data/Data_HNscore/HS_mouse_meta-data.csv",
    separator=","
).with_columns( # Cell-line(type) が null の場合は tissue を sample として使用し、統合する
    pl.when(pl.col("Cell-line(type)").is_not_null())
    .then(pl.col("Cell-line(type)"))
    .otherwise(pl.col("tissue"))
    .alias("sample")
).with_columns([
    pl.col("sample")
        .str.replace_all(",", " ")  # , to space
        .str.replace_all(r"\s+", " ")  # replace continuous spaces with one space
        .str.replace_all(r"[()]", "")  # remove parentheses (  )
        .str.replace_all(r"\s+$", "")  # remove trailing spaces ('\s+' matches any whitespace character, and '$' matches the end of the string)
        .str.replace_all(" ", "-")  # replace remaining spaces with hyphen
    ]
)

display(metadata)

SRA study count,SRA Study,pair,Source of Stress (Condition),Control,strategy,LibrarySelect,temperature,time,treatment,control temperature,Cell-line(type),tissue,strain,Genotype/Phenotype,LibraryLayout,RNA,Instrument,note,sample
i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""SRP212072""",1,"""SRR9601002""","""SRR9600999""","""RNA-seq""","""cDNA""","""45℃""","""1h.""","""culture incubator (Thermo Fish…","""37℃""","""B16-OVA(mouse melanoma cell)""",null,"""C57BL/6J(strain)""",null,"""PAIRED""",null,"""Illumina NovaSeq 6000""","""control siRNA""","""B16-OVAmouse-melanoma-cell"""
null,null,2,"""SRR9601003""","""SRR9601000""","""RNA-seq""","""cDNA""","""45℃""","""1h.""","""culture incubator (Thermo Fish…","""37℃""","""B16-OVA(mouse melanoma cell)""",null,"""C57BL/6J(strain)""",null,"""PAIRED""",null,"""Illumina NovaSeq 6000""","""control siRNA""","""B16-OVAmouse-melanoma-cell"""
null,null,3,"""SRR9601008""","""SRR9601004""","""RNA-seq""","""cDNA""","""45℃""","""1h.""","""culture incubator (Thermo Fish…","""37℃""","""B16-OVA(mouse melanoma cell)""",null,"""C57BL/6J(strain)""",null,"""PAIRED""",null,"""Illumina NovaSeq 6000""","""pool of siRNAs against YAP and…","""B16-OVAmouse-melanoma-cell"""
null,null,4,"""SRR9601009""","""SRR9601005""","""RNA-seq""","""cDNA""","""45℃""","""1h.""","""culture incubator (Thermo Fish…","""37℃""","""B16-OVA(mouse melanoma cell)""",null,"""C57BL/6J(strain)""",null,"""PAIRED""",null,"""Illumina NovaSeq 6000""","""pool of siRNAs against YAP and…","""B16-OVAmouse-melanoma-cell"""
null,null,5,"""SRR9601010""","""SRR9601006""","""RNA-seq""","""cDNA""","""45℃""","""1h.""","""culture incubator (Thermo Fish…","""37℃""","""B16-OVA(mouse melanoma cell)""",null,"""C57BL/6J(strain)""",null,"""PAIRED""",null,"""Illumina NovaSeq 6000""","""pool of siRNAs against YAP and…","""B16-OVAmouse-melanoma-cell"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,null,238,"""SRR11684186""","""SRR11684195""","""RNA-seq""","""cDNA""","""42℃""","""20min.""",null,"""33℃""",null,"""Testis""","""C57BL6""","""Mageb4 KO""","""PAIRED""",null,"""Illumina HiSeq 4000""","""Mageb4 KO 72h after heat shock…","""Testis"""
30,"""SRP151770""",239,"""SRR7463322""","""SRR7463318""","""RNA-seq""","""cDNA""","""43℃""","""30min.""","""water bath""","""37℃""",null,"""Vestibular system tissue (Utri…","""CBA/J""",null,"""PAIRED""",null,"""Illumina HiSeq 1500""","""Adult mause (P30-P60), 2h reco…","""Vestibular-system-tissue-Utric…"
null,null,240,"""SRR7463323""","""SRR7463319""","""RNA-seq""","""cDNA""","""43℃""","""30min.""","""water bath""","""37℃""",null,"""Vestibular system tissue (Utri…","""CBA/J""",null,"""PAIRED""",null,"""Illumina HiSeq 1500""","""Adult mause (P30-P60), 2h reco…","""Vestibular-system-tissue-Utric…"


In [5]:
HNratio = pl.read_csv(
    "../Data/Data_HNscore/HN-ratio_mouse_all.csv", 
    separator=","
).rename(
    {
        "GeneName": "GENEID"
    }
)


display(HNratio)

GENEID,SRR8835898,SRR8835908,SRR8835910,SRR8835917,SRR8835894,SRR8835928,SRR8835900,SRR8835904,SRR8835891,SRR8835916,SRR8835920,SRR8835935,SRR2068908,SRR2068909,SRR2068912,SRR2068913,SRR8494667,SRR8494669,SRR8494671,SRR8494673,SRR8494675,SRR8494677,SRR8494679,SRR8494681,SRR8494691,SRR8494693,SRR8494695,SRR8494697,SRR8494699,SRR8494701,SRR8494703,SRR8494705,SRR7092927,SRR7092929,SRR6439785,SRR6439787,…,SRR8713378,SRR8713379,SRR8713368,SRR8713369,SRR8713370,SRR2144180,SRR2144181,SRR2144182,SRR2144183,SRR2144184,SRR2144185,SRR2144188,SRR2144189,SRR2144190,SRR2144191,SRR2144192,SRR2144193,SRR2144196,SRR2144197,SRR2144198,SRR2144199,SRR2144200,SRR2144201,SRR7458429,SRR6262112,SRR6262113,SRR6262116,SRR6262117,SRR6262104,SRR6262105,SRR6262108,SRR6262109,SRR13571065,SRR13571066,SRR13571067,SRR3624807,SRR3624808
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""0610009B22Rik""",0.888477,0.472901,0.585592,1.348745,0.704214,1.117431,0.731358,1.519465,1.117518,0.669057,0.93911,85.817063,0.683529,1.298941,1.171331,1.090664,3.972982,0.812053,1.451236,1.703709,4.024009,0.632686,0.899325,1.003064,0.781833,0.533815,0.594683,0.897912,0.86295,0.529685,0.727475,1.640231,0.704483,0.985401,1.506928,0.921028,…,0.383905,0.412896,2.652033,1.102511,1.32436,0.961777,1.379101,1.669197,1.870865,1.14608,0.323536,0.171554,0.110785,1.069491,0.804621,1.319187,1.681715,0.454974,0.36019,1.244047,1.56152,2.376479,0.917361,1.21097,4.522056,0.625339,1.475054,4.652027,0.533154,1.523169,3.681419,1.290168,1.0,1.0,0.220645,1.515786,1.264775
"""0610010F05Rik""",1.301893,0.993249,1.09083,1.752709,1.050022,0.740508,0.682207,0.8303,1.085242,0.866288,0.591468,15.849933,0.560752,1.025288,0.400601,0.535501,5.144965,1.154868,1.170252,0.726107,3.947539,1.440232,1.060984,0.52689,0.779771,0.811449,1.030146,1.492635,1.17172,1.518397,0.314611,0.566165,0.938511,1.785951,1.294,0.625073,…,2.930904,3.840904,2.808893,2.087653,2.51252,0.993413,0.968014,0.907144,0.673175,1.130497,0.131761,0.135847,0.084142,0.39776,0.879705,0.175334,0.470106,0.521215,0.574091,1.313915,2.045847,3.026641,1.201071,3.137886,0.922749,0.952688,1.490413,3.486095,1.030767,2.252301,3.749367,2.454809,3.99786,1.0,7.577615,1.197426,0.745424
"""0610010K14Rik""",1.633539,0.318482,1.338232,1.319906,0.724082,0.865305,1.264106,0.721409,0.853692,0.831326,0.56246,38.462246,0.752448,0.81768,0.654543,0.767625,1.440976,0.934084,1.28849,2.238425,2.591539,1.421317,1.197143,1.303096,0.659862,0.470973,1.10723,0.856258,0.926049,0.758249,1.883928,1.692645,0.868496,0.905947,1.372529,0.700447,…,2.525657,4.993299,0.138346,1.894906,1.212691,0.784016,3.765472,1.417482,2.660671,2.489764,0.061067,0.122587,0.063433,0.561869,1.114533,0.302764,0.847027,0.10486,0.198684,1.017632,1.573518,4.862532,0.990388,2.190758,0.945129,1.410564,3.398298,3.138838,1.568461,1.842289,4.277982,1.101588,1.0,1.0,1.0,0.889799,0.451231
"""0610012G03Rik""",1.250494,0.584869,0.65624,1.862847,0.672534,1.072469,0.93719,1.019688,1.160474,0.888001,0.862622,44.619135,1.20957,1.084018,0.874951,1.16311,3.893803,1.00418,1.094192,1.09372,3.786643,0.936483,0.850894,0.878284,0.884353,0.670857,0.888402,1.076101,0.951451,0.911551,1.301394,1.298785,1.020867,0.918052,1.547709,0.936264,…,0.593911,0.869913,1.763948,1.039949,0.690518,0.645407,0.959884,0.788437,1.250749,0.479205,0.114527,0.175591,0.126335,0.410451,0.755652,0.27686,0.544098,0.417934,0.435387,1.295517,1.573813,2.122747,0.908573,1.038196,1.24298,1.100001,1.630976,2.65972,0.805139,1.393825,3.264708,1.688933,1.0,1.0,1.0,0.704087,0.848764
"""0610030E20Rik""",1.021152,0.592594,0.937276,1.702173,0.969005,0.994161,0.79712,1.067343,1.102271,0.818644,0.740996,14.696016,1.271687,1.676788,1.217478,1.057126,3.407591,0.987954,1.230204,1.284855,3.440074,1.027005,0.999894,0.956125,1.048658

In [6]:
rename_dict = dict(
    zip(
        metadata['Source of Stress (Condition)'], metadata['sample']
    )
)

print(rename_dict)

# change the column name based on the rename_dict
new_columns = []
seen = set()
for col in HNratio.columns:
    new_col = rename_dict.get(col, col)
    if new_col in seen:
        suffix = 1
        while f"{new_col}_{suffix}" in seen:
            suffix += 1
        new_col = f"{new_col}_{suffix}"
    seen.add(new_col)
    new_columns.append(new_col)

HNratio = HNratio.rename(
    dict(
        zip(
            HNratio.columns, new_columns
        )
    )
)

display(HNratio)

{'SRR9601002': 'B16-OVAmouse-melanoma-cell', 'SRR9601003': 'B16-OVAmouse-melanoma-cell', 'SRR9601008': 'B16-OVAmouse-melanoma-cell', 'SRR9601009': 'B16-OVAmouse-melanoma-cell', 'SRR9601010': 'B16-OVAmouse-melanoma-cell', 'SRR2068908': 'Splenic-B220+-cells', 'SRR2068909': 'Splenic-B220+-cells', 'SRR2068912': 'Splenic-B220+-cells', 'SRR2068913': 'Splenic-B220+-cells', 'SRR12044994': 'Mouse-splenic-CD4+-T-cells', 'SRR12044995': 'Mouse-splenic-CD4+-T-cells', 'SRR12044996': 'Mouse-splenic-CD4+-T-cells', 'SRR14119173': 'CD8-T-cell', 'SRR14119174': 'CD8-T-cell', 'SRR14119175': 'CD8-T-cell', 'SRR6439785': 'MEF-cells', 'SRR6439787': 'MEF-cells', 'SRR13125320': 'Mouse-embryonic-fibroblasts', 'SRR13125321': 'Mouse-embryonic-fibroblasts', 'SRR13125322': 'Mouse-embryonic-fibroblasts', 'SRR13125326': 'Mouse-embryonic-fibroblasts', 'SRR13125327': 'Mouse-embryonic-fibroblasts', 'SRR13125328': 'Mouse-embryonic-fibroblasts', 'SRR5558715': 'mouse-embryonic-fibroblasts', 'SRR5558716': 'mouse-embryonic-fib

GENEID,kidney,kidney_1,kidney_2,kidney_3,kidney_4,kidney_5,kidney_6,kidney_7,kidney_8,kidney_9,kidney_10,kidney_11,Splenic-B220+-cells,Splenic-B220+-cells_1,Splenic-B220+-cells_2,Splenic-B220+-cells_3,mESC,mESC_1,mESC_2,mESC_3,mESC_4,mESC_5,mESC_6,mESC_7,mESC_8,mESC_9,mESC_10,mESC_11,mESC_12,mESC_13,mESC_14,mESC_15,cortexbrain,cortexbrain_1,MEF-cells,MEF-cells_1,…,MEF_10,MEF_11,MEF_12,MEF_13,MEF_14,MEF-cells_2,MEF-cells_3,MEF-cells_4,MEF-cells_5,MEF-cells_6,MEF-cells_7,MEF-cells_8,MEF-cells_9,MEF-cells_10,MEF-cells_11,MEF-cells_12,MEF-cells_13,MEF-cells_14,MEF-cells_15,MEF-cells_16,MEF-cells_17,MEF-cells_18,MEF-cells_19,MEF-cells_20,immortalized-MEFs,immortalized-MEFs_1,immortalized-MEFs_2,immortalized-MEFs_3,immortalized-MEFs_4,immortalized-MEFs_5,immortalized-MEFs_6,immortalized-MEFs_7,Testis_12,Testis_13,Testis_14,NIH/3T3-Fibroblasts,NIH/3T3-Fibroblasts_1
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""0610009B22Rik""",0.888477,0.472901,0.585592,1.348745,0.704214,1.117431,0.731358,1.519465,1.117518,0.669057,0.93911,85.817063,0.683529,1.298941,1.171331,1.090664,3.972982,0.812053,1.451236,1.703709,4.024009,0.632686,0.899325,1.003064,0.781833,0.533815,0.594683,0.897912,0.86295,0.529685,0.727475,1.640231,0.704483,0.985401,1.506928,0.921028,…,0.383905,0.412896,2.652033,1.102511,1.32436,0.961777,1.379101,1.669197,1.870865,1.14608,0.323536,0.171554,0.110785,1.069491,0.804621,1.319187,1.681715,0.454974,0.36019,1.244047,1.56152,2.376479,0.917361,1.21097,4.522056,0.625339,1.475054,4.652027,0.533154,1.523169,3.681419,1.290168,1.0,1.0,0.220645,1.515786,1.264775
"""0610010F05Rik""",1.301893,0.993249,1.09083,1.752709,1.050022,0.740508,0.682207,0.8303,1.085242,0.866288,0.591468,15.849933,0.560752,1.025288,0.400601,0.535501,5.144965,1.154868,1.170252,0.726107,3.947539,1.440232,1.060984,0.52689,0.779771,0.811449,1.030146,1.492635,1.17172,1.518397,0.314611,0.566165,0.938511,1.785951,1.294,0.625073,…,2.930904,3.840904,2.808893,2.087653,2.51252,0.993413,0.968014,0.907144,0.673175,1.130497,0.131761,0.135847,0.084142,0.39776,0.879705,0.175334,0.470106,0.521215,0.574091,1.313915,2.045847,3.026641,1.201071,3.137886,0.922749,0.952688,1.490413,3.486095,1.030767,2.252301,3.749367,2.454809,3.99786,1.0,7.577615,1.197426,0.745424
"""0610010K14Rik""",1.633539,0.318482,1.338232,1.319906,0.724082,0.865305,1.264106,0.721409,0.853692,0.831326,0.56246,38.462246,0.752448,0.81768,0.654543,0.767625,1.440976,0.934084,1.28849,2.238425,2.591539,1.421317,1.197143,1.303096,0.659862,0.470973,1.10723,0.856258,0.926049,0.758249,1.883928,1.692645,0.868496,0.905947,1.372529,0.700447,…,2.525657,4.993299,0.138346,1.894906,1.212691,0.784016,3.765472,1.417482,2.660671,2.489764,0.061067,0.122587,0.063433,0.561869,1.114533,0.302764,0.847027,0.10486,0.198684,1.017632,1.573518,4.862532,0.990388,2.190758,0.945129,1.410564,3.398298,3.138838,1.568461,1.842289,4.277982,1.101588,1.0,1.0,1.0,0.889799,0.451231
"""0610012G03Rik""",1.250494,0.584869,0.65624,1.862847,0.672534,1.072469,0.93719,1.019688,1.160474,0.888001,0.862622,44.619135,1.20957,1.084018,0.874951,1.16311,3.893803,1.00418,1.094192,1.09372,3.786643,0.936483,0.850894,0.878284,0.884353,0.670857,0.888402,1.076101,0.951451,0.911551,1.301394,1.298785,1.020867,0.918052,1.547709,0.936264,…,0.593911,0.869913,1.763948,1.039949,0.690518,0.645407,0.959884,0.788437,1.250749,0.479205,0.114527,0.175591,0.126335,0.410451,0.755652,0.27686,0.544098,0.417934,0.435387,1.295517,1.573813,2.122747,0.908573,1.038196,1.24298,1.100001,1.630976,2.65972,0.805139,1.393825,3.264708,1.688933,1.0,1.0,1.0,0.704087,0.848764
"""0610030E20Rik""",1.021152,0.592594,0.937276,1.702173,0.969005,0.994161,0.79712,1.067343,1.102271,0.818644,0.740996,14.696016,1.271687,1.676788,1.217478,1.057126,3.407591,0.987954,1.2302

In [7]:
# define a function to categorize the HNratio
def categorize(x):
    if x >= threshold:
        return 'UP'
    elif x <= 1 / threshold:
        return 'DOWN'
    else:
        return 'UNCHANGED'

# apply the categorize function to all columns except GENEID
result = HNratio.with_columns([
    pl.col(col).map_elements(categorize, return_dtype=pl.String)
    for col in HNratio.columns if col != 'GENEID'
])

# display the result
display(result)

GENEID,kidney,kidney_1,kidney_2,kidney_3,kidney_4,kidney_5,kidney_6,kidney_7,kidney_8,kidney_9,kidney_10,kidney_11,Splenic-B220+-cells,Splenic-B220+-cells_1,Splenic-B220+-cells_2,Splenic-B220+-cells_3,mESC,mESC_1,mESC_2,mESC_3,mESC_4,mESC_5,mESC_6,mESC_7,mESC_8,mESC_9,mESC_10,mESC_11,mESC_12,mESC_13,mESC_14,mESC_15,cortexbrain,cortexbrain_1,MEF-cells,MEF-cells_1,…,MEF_10,MEF_11,MEF_12,MEF_13,MEF_14,MEF-cells_2,MEF-cells_3,MEF-cells_4,MEF-cells_5,MEF-cells_6,MEF-cells_7,MEF-cells_8,MEF-cells_9,MEF-cells_10,MEF-cells_11,MEF-cells_12,MEF-cells_13,MEF-cells_14,MEF-cells_15,MEF-cells_16,MEF-cells_17,MEF-cells_18,MEF-cells_19,MEF-cells_20,immortalized-MEFs,immortalized-MEFs_1,immortalized-MEFs_2,immortalized-MEFs_3,immortalized-MEFs_4,immortalized-MEFs_5,immortalized-MEFs_6,immortalized-MEFs_7,Testis_12,Testis_13,Testis_14,NIH/3T3-Fibroblasts,NIH/3T3-Fibroblasts_1
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""0610009B22Rik""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UP""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""",…,"""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""DOWN""","""DOWN""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED"""
"""0610010F05Rik""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UP""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UP""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""",…,"""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""DOWN""","""DOWN""","""DOWN""","""UNCHANGED""","""UNCHANGED""","""DOWN""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UP""","""UNCHANGED""","""UNCHANGED"""
"""0610010K14Rik""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UP""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""",

In [8]:
# remove the suffix from the column name
def remove_suffix(col_name):
    return col_name.rsplit('_', 1)[0]


# get the sample column list
sample_columns = set(remove_suffix(col) for col in result.columns if col != 'GENEID')

print(sample_columns)

counts = []
for sample in sample_columns:
    # get the dataframe for each sample
    sample_df = result.select(
        ['GENEID'] + [col for col in result.columns if remove_suffix(col) == sample]
    )
    # count the number of UP, DOWN, UNCHANGED
    count = sample_df.with_columns([
        (pl.sum_horizontal([
            (pl.col(col) == "UP").cast(pl.Int32) for col in sample_df.columns if col != "GENEID"
        ])).alias(f"UP({sample})"),
        (pl.sum_horizontal([
            (pl.col(col) == "DOWN").cast(pl.Int32) for col in sample_df.columns if col != "GENEID"
        ])).alias(f"DOWN({sample})"),
        pl.sum_horizontal([
            (pl.col(col) == "UNCHANGED").cast(pl.Int32) for col in sample_df.columns if col != "GENEID"
        ]).alias(f"UNCHANGED({sample})")
    ])
    # append the count to the counts list
    counts.append(count)

# combine all samples
final_count = counts[0] # initialize the final_count with the first count
for count in counts[1:]: # [1:] is the list of counts from the second sample to the last sample
    final_count = final_count.join(
        count, 
        on=['GENEID'], 
        how='left',
        coalesce=True
    )

final_count = final_count.select(
    ['GENEID'] + [col for col in final_count.columns if col.startswith('UP(') or col.startswith('DOWN(') or col.startswith('UNCHANGED(')]
)

display(final_count)

{'immortalized-MEFs', 'mESC', 'quadriceps-femoris-muscle', 'Splenic-B220+-cells', 'mammary-gland', 'Mouse-splenic-CD4+-T-cells', 'CD8-T-cell', '3T3-cells', 'Vestibular-system-tissue-Utricle', 'cortexbrain', 'mouse-embryonic-fibroblasts', 'Mouse-testicular-peritubular-myoid-PTM-cells', 'NIH/3T3-cells', 'pituitary', 'kidney', 'B16-OVAmouse-melanoma-cell', 'Fetal-jejunum', 'Testis', 'Raw264.7.', 'hypothalamus', 'MEF', 'Fetal-duodenum', 'Cultured-utricles', 'Mouse-embryonic-fibroblasts', 'MEF-cells', 'Placenta', 'NIH/3T3-Fibroblasts', 'Th17-cells', 'NIH3T3', 'mouse-embryonic-stem-cells'}


GENEID,UP(immortalized-MEFs),DOWN(immortalized-MEFs),UNCHANGED(immortalized-MEFs),UP(mESC),DOWN(mESC),UNCHANGED(mESC),UP(quadriceps-femoris-muscle),DOWN(quadriceps-femoris-muscle),UNCHANGED(quadriceps-femoris-muscle),UP(Splenic-B220+-cells),DOWN(Splenic-B220+-cells),UNCHANGED(Splenic-B220+-cells),UP(mammary-gland),DOWN(mammary-gland),UNCHANGED(mammary-gland),UP(Mouse-splenic-CD4+-T-cells),DOWN(Mouse-splenic-CD4+-T-cells),UNCHANGED(Mouse-splenic-CD4+-T-cells),UP(CD8-T-cell),DOWN(CD8-T-cell),UNCHANGED(CD8-T-cell),UP(3T3-cells),DOWN(3T3-cells),UNCHANGED(3T3-cells),UP(Vestibular-system-tissue-Utricle),DOWN(Vestibular-system-tissue-Utricle),UNCHANGED(Vestibular-system-tissue-Utricle),UP(cortexbrain),DOWN(cortexbrain),UNCHANGED(cortexbrain),UP(mouse-embryonic-fibroblasts),DOWN(mouse-embryonic-fibroblasts),UNCHANGED(mouse-embryonic-fibroblasts),UP(Mouse-testicular-peritubular-myoid-PTM-cells),DOWN(Mouse-testicular-peritubular-myoid-PTM-cells),UNCHANGED(Mouse-testicular-peritubular-myoid-PTM-cells),…,UNCHANGED(Testis),UP(Raw264.7.),DOWN(Raw264.7.),UNCHANGED(Raw264.7.),UP(hypothalamus),DOWN(hypothalamus),UNCHANGED(hypothalamus),UP(MEF),DOWN(MEF),UNCHANGED(MEF),UP(Fetal-duodenum),DOWN(Fetal-duodenum),UNCHANGED(Fetal-duodenum),UP(Cultured-utricles),DOWN(Cultured-utricles),UNCHANGED(Cultured-utricles),UP(Mouse-embryonic-fibroblasts),DOWN(Mouse-embryonic-fibroblasts),UNCHANGED(Mouse-embryonic-fibroblasts),UP(MEF-cells),DOWN(MEF-cells),UNCHANGED(MEF-cells),UP(Placenta),DOWN(Placenta),UNCHANGED(Placenta),UP(NIH/3T3-Fibroblasts),DOWN(NIH/3T3-Fibroblasts),UNCHANGED(NIH/3T3-Fibroblasts),UP(Th17-cells),DOWN(Th17-cells),UNCHANGED(Th17-cells),UP(NIH3T3),DOWN(NIH3T3),UNCHANGED(NIH3T3),UP(mouse-embryonic-stem-cells),DOWN(mouse-embryonic-stem-cells),UNCHANGED(mouse-embryonic-stem-cells)
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""0610009B22Rik""",0,0,8,0,0,24,0,0,6,0,0,4,0,0,8,0,0,3,0,0,3,0,1,3,0,0,4,0,0,2,0,0,2,0,0,12,…,15,0,0,3,0,0,8,0,2,13,0,0,3,0,0,17,0,0,6,0,2,19,0,0,3,0,0,2,0,0,2,0,1,9,0,0,16
"""0610010F05Rik""",0,0,8,1,0,23,0,0,6,0,0,4,0,0,8,0,0,3,0,0,3,0,2,2,0,0,4,0,0,2,0,0,2,0,0,12,…,14,0,0,3,0,0,8,0,0,15,0,0,3,0,0,17,0,0,6,0,4,17,0,0,3,0,0,2,0,0,2,2,0,8,0,0,16
"""0610010K14Rik""",0,0,8,0,0,24,0,0,6,0,0,4,0,0,8,0,0,3,0,0,3,1,0,3,0,0,4,0,0,2,0,0,2,0,0,12,…,15,0,0,3,0,0,8,2,1,12,0,0,3,0,0,17,0,0,6,0,5,16,0,0,3,0,0,2,0,0,2,0,0,10,1,0,15
"""0610012G03Rik""",0,0,8,0,0,24,0,0,6,0,0,4,0,0,8,0,0,3,0,0,3,0,1,3,0,0,4,0,0,2,0,0,2,0,0,12,…,15,0,0,3,0,0,8,0,2,13,0,0,3,0,0,17,0,0,6,0,3,18,0,0,3,0,0,2,0,0,2,0,0,10,1,0,15
"""0610030E20Rik""",0,0,8,0,0,24,0,0,6,0,0,4,0,0,8,0,0,3,0,0,3,0,0,4,0,0,4,0,0,2,0,0,2,0,0,12,…,13,0,0,3,0,0,8,0,3,12,0,0,3,0,0,17,0,0,6,0,2,19,0,0,3,0,0,2,0,0,2,0,0,10,0,0,16
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Zyg11a""",0,0,8,0,0,24,0,0,6,0,0,4,0,0,8,0,0,3,0,0,3,0,0,4,0,0,4,0,0,2,0,0,2,0,0,12,…,15,0,0,3,0,0,8,0,0,15,0,0,3,0,0,17,0,0,6,0,2,19,1,0,2,0,0,2,0,0,2,0,0,10,0,0,16
"""Zyg11b""",0,0,8,0,0,24,0,0,6,0,0,4,0,0,8,0,0,3,0,0,3,0,0,4,0,0,4,0,0,2,0,0,2,0,0,12,…,15,0,0,3,0,0,8,0,1,14,0,0,3,0,1,16,0,0,6,0,2,19,0,0,3,0,0,2,0,0,2,0,0,10,0,0,16
"""Zyx""",0,0,8,0,0,24,0,0,6,0,0,4,0,0,8,0,0,3,0,0,3,0,1,3,0,0,4,0,0,2,0,0,2,0,0,12,…,9,0,0,3,0,0,8,0,1,14,0,0,3,0,0,17,0,0,6,0,2,19,0,0,3,0,0,2,0,0,2,0,0,10,0,0,16


In [9]:
# get the sample name list
sample_names = [col.split('(')[1].split(')')[0] for col in final_count.columns if col.startswith('UP(')]
print(sample_names)

# calculate HNscore for each sample
HNscore_sample_count = final_count.with_columns([
    (
        pl.col(f'UP({sample})') - pl.col(f'DOWN({sample})')
    ).alias(
        f'HN{threshold}({sample})'
    )
    for sample in sample_names # sample_names is the list of sample names
])

# select the columns that start with HN and end with the sample name
HNscore_sample_count = HNscore_sample_count.select(
    ['GENEID'] + 
    [col for col in HNscore_sample_count.columns if col.startswith(f'HN{threshold}(') and col.endswith(')')]
).with_columns(
    pl.sum_horizontal([
        pl.col(col) for col in HNscore_sample_count.columns if col.startswith(f'HN{threshold}(') and col.endswith(')')
    ]).alias(f'HN{threshold}(all)')
).sort(
    by=[f'HN{threshold}(all)'],
    descending=True
)

display(HNscore_sample_count)

['immortalized-MEFs', 'mESC', 'quadriceps-femoris-muscle', 'Splenic-B220+-cells', 'mammary-gland', 'Mouse-splenic-CD4+-T-cells', 'CD8-T-cell', '3T3-cells', 'Vestibular-system-tissue-Utricle', 'cortexbrain', 'mouse-embryonic-fibroblasts', 'Mouse-testicular-peritubular-myoid-PTM-cells', 'NIH/3T3-cells', 'pituitary', 'kidney', 'B16-OVAmouse-melanoma-cell', 'Fetal-jejunum', 'Testis', 'Raw264.7.', 'hypothalamus', 'MEF', 'Fetal-duodenum', 'Cultured-utricles', 'Mouse-embryonic-fibroblasts', 'MEF-cells', 'Placenta', 'NIH/3T3-Fibroblasts', 'Th17-cells', 'NIH3T3', 'mouse-embryonic-stem-cells']


GENEID,HN5(immortalized-MEFs),HN5(mESC),HN5(quadriceps-femoris-muscle),HN5(Splenic-B220+-cells),HN5(mammary-gland),HN5(Mouse-splenic-CD4+-T-cells),HN5(CD8-T-cell),HN5(3T3-cells),HN5(Vestibular-system-tissue-Utricle),HN5(cortexbrain),HN5(mouse-embryonic-fibroblasts),HN5(Mouse-testicular-peritubular-myoid-PTM-cells),HN5(NIH/3T3-cells),HN5(pituitary),HN5(kidney),HN5(B16-OVAmouse-melanoma-cell),HN5(Fetal-jejunum),HN5(Testis),HN5(Raw264.7.),HN5(hypothalamus),HN5(MEF),HN5(Fetal-duodenum),HN5(Cultured-utricles),HN5(Mouse-embryonic-fibroblasts),HN5(MEF-cells),HN5(Placenta),HN5(NIH/3T3-Fibroblasts),HN5(Th17-cells),HN5(NIH3T3),HN5(mouse-embryonic-stem-cells),HN5(all)
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""Hspa1b""",8,17,4,2,8,0,2,4,4,0,2,3,2,8,14,5,1,4,0,6,13,1,17,6,5,0,2,0,10,1,149
"""Hspa1a""",0,17,4,3,8,0,1,4,4,0,2,5,2,8,15,5,1,3,0,7,10,1,17,5,4,0,1,0,10,0,137
"""Hspb1""",2,2,2,3,5,1,1,3,4,-1,2,3,2,6,18,5,0,8,0,4,11,0,16,0,4,0,0,0,9,1,111
"""Gm20481""",3,9,0,3,4,0,0,2,4,0,1,3,1,6,15,3,-1,2,0,2,5,0,17,3,1,-1,0,1,5,0,88
"""Atf3""",5,2,1,2,0,0,0,0,3,0,2,3,0,1,16,2,0,10,0,1,6,0,8,2,5,0,0,1,8,0,78
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Il4i1""",0,-5,0,-4,2,0,0,0,0,0,0,-1,-2,-2,0,-1,1,-6,0,-4,0,0,-1,0,-5,0,0,0,-2,0,-30
"""Spata45""",0,-1,-1,2,0,-1,0,-2,0,0,-2,-2,-1,-1,0,-3,-1,-6,0,-3,-2,-1,-4,-1,-2,0,0,-2,3,0,-31
"""Tmem89""",0,-2,-1,-1,0,0,0,0,0,0,-1,0,-2,-1,0,-1,-1,-7,0,-1,-7,1,-1,0,-5,0,0,0,-1,0,-31


&nbsp;

&nbsp;

## HN-score validation

In [10]:
# select HN5 columns (exclude HN5(all))
hn5_columns = [col for col in HNscore_sample_count.columns if col.startswith("HN5(") and col != "HN5(all)"]
display(hn5_columns)

['HN5(immortalized-MEFs)',
 'HN5(mESC)',
 'HN5(quadriceps-femoris-muscle)',
 'HN5(Splenic-B220+-cells)',
 'HN5(mammary-gland)',
 'HN5(Mouse-splenic-CD4+-T-cells)',
 'HN5(CD8-T-cell)',
 'HN5(3T3-cells)',
 'HN5(Vestibular-system-tissue-Utricle)',
 'HN5(cortexbrain)',
 'HN5(mouse-embryonic-fibroblasts)',
 'HN5(Mouse-testicular-peritubular-myoid-PTM-cells)',
 'HN5(NIH/3T3-cells)',
 'HN5(pituitary)',
 'HN5(kidney)',
 'HN5(B16-OVAmouse-melanoma-cell)',
 'HN5(Fetal-jejunum)',
 'HN5(Testis)',
 'HN5(Raw264.7.)',
 'HN5(hypothalamus)',
 'HN5(MEF)',
 'HN5(Fetal-duodenum)',
 'HN5(Cultured-utricles)',
 'HN5(Mouse-embryonic-fibroblasts)',
 'HN5(MEF-cells)',
 'HN5(Placenta)',
 'HN5(NIH/3T3-Fibroblasts)',
 'HN5(Th17-cells)',
 'HN5(NIH3T3)',
 'HN5(mouse-embryonic-stem-cells)']

In [11]:
# count positive and negative samples
HNscore_sample_validate = HNscore_sample_count.with_columns([
    pl.fold(0, lambda acc, x: acc + (x > 0).cast(pl.Int32), hn5_columns).alias("positive_samples_count_mouse"),
    pl.fold(0, lambda acc, x: acc + (x < 0).cast(pl.Int32), hn5_columns).alias("negative_samples_count_mouse")
])

# assign tag to the contribution of samples
def tag_contribution(count, type, hn5_column):
    total_samples = len(hn5_column)
    if count == total_samples:
        return f"all samples {type} contribute to the score"
    else:
        return f"{count} / {total_samples} samples {type} contribute to the score"
    
HNscore_sample_validate = HNscore_sample_validate.with_columns([
    pl.lit(len(hn5_columns)).alias("total_samples"),
    pl.col("positive_samples_count_mouse").map_elements(lambda x: tag_contribution(x, "positively", hn5_columns), return_dtype=pl.Utf8).alias("positive_contribution_mouse"),
    pl.col("negative_samples_count_mouse").map_elements(lambda x: tag_contribution(x, "negatively", hn5_columns), return_dtype=pl.Utf8).alias("negative_contribution_mouse")
]
).sort(
    by=["HN5(all)"],
    descending=True
).drop(
    hn5_columns
)

# save the result to csv
HNscore_sample_validate.write_csv(
    f'../Data/Data_HNscore/HNscore_sample_validate_HN{threshold}_mouse.tsv',
    separator='\t'
)

display(HNscore_sample_validate.head())

GENEID,HN5(all),positive_samples_count_mouse,negative_samples_count_mouse,total_samples,positive_contribution_mouse,negative_contribution_mouse
str,i32,i32,i32,i32,str,str
"""Hspa1b""",149,25,0,30,"""25 / 30 samples positively con…","""0 / 30 samples negatively cont…"
"""Hspa1a""",137,23,0,30,"""23 / 30 samples positively con…","""0 / 30 samples negatively cont…"
"""Hspb1""",111,22,1,30,"""22 / 30 samples positively con…","""1 / 30 samples negatively cont…"
"""Gm20481""",88,20,2,30,"""20 / 30 samples positively con…","""2 / 30 samples negatively cont…"
"""Atf3""",78,18,0,30,"""18 / 30 samples positively con…","""0 / 30 samples negatively cont…"


&nbsp;

&nbsp;


## Convert check

- Ensembl Biomartでヒトと互換性があるか確認しておく
- TogoIDでヒットしたヒトのUniProt IDをキーにMGI IDを取得しておく

In [12]:
!bash ../scripts/togoid_convert.sh ../out/rice_up/uniprot_id_human_up.txt ../out/rice_up/ uniprot,ensembl_protein,ensembl_transcript,ensembl_gene,hgnc,mgi_gene rice_up_togoid_convert_to_mouse.tsv
!bash ../scripts/togoid_convert.sh ../out/rice_down/uniprot_id_human_down.txt ../out/rice_down/ uniprot,ensembl_protein,ensembl_transcript,ensembl_gene,hgnc,mgi_gene rice_down_togoid_convert_to_mouse.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60262  100 49160  100 11102  12537   2831  0:00:03  0:00:03 --:--:-- 15365
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 58246  100 49644  100  8602  27749   4808  0:00:01  0:00:01 --:--:-- 32539
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 51111  100 44009  100  7102  28595   4614  0:00:01  0:00:01 --:--:-- 33210
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  154k  100  147k  100  7102  48808   2294  0:00:03  0:00:03 --:--:-- 51102
  % Total    % Received % Xferd  Average Speed   Tim

In [13]:
togoid_convert_to_mouse_up = pl.read_csv(
    "../out/rice_up/rice_up_togoid_convert_to_mouse.tsv",
    separator="\t"
).filter(
    pl.col("hgnc_id").is_not_null()
)

display(togoid_convert_to_mouse_up)

uniprot_id,ensembl_protein_id,ensembl_transcript_id,ensembl_gene_id,hgnc_id,mgi_gene_id
str,str,str,str,i64,i64
"""A0A024QZW4""","""ENSP00000518269""","""ENST00000710437""","""ENSG00000124786""",21601,1913978
"""A0A024QZW4""","""ENSP00000498054""","""ENST00000648987""","""ENSG00000124786""",21601,1913978
"""A0A024R3B9""","""ENSP00000435411""","""ENST00000525823""","""ENSG00000109846""",2389,88516
"""A0A024R3B9""","""ENSP00000435960""","""ENST00000528961""","""ENSG00000109846""",2389,88516
"""A0A024R3B9""","""ENSP00000435046""","""ENST00000533280""","""ENSG00000109846""",2389,88516
…,…,…,…,…,…
"""X6RA14""","""ENSP00000367969""","""ENST00000378697""","""ENSG00000139684""",3465,95421
"""X6RBJ8""","""ENSP00000380187""","""ENST00000396990""","""ENSG00000198876""",19911,1916220
"""X6RHX1""","""ENSP00000407993""","""ENST00000423670""","""ENSG00000000457""",19285,1921385


In [14]:
togoid_convert_to_mouse_down = pl.read_csv(
    "../out/rice_down/rice_down_togoid_convert_to_mouse.tsv",
    separator="\t"
).filter(
    pl.col("hgnc_id").is_not_null()
)

display(togoid_convert_to_mouse_down)

uniprot_id,ensembl_protein_id,ensembl_transcript_id,ensembl_gene_id,hgnc_id,mgi_gene_id
str,str,str,str,i64,i64
"""A0A024QZP7""","""ENSP00000397973""","""ENST00000448257""","""ENSG00000170312""",1722,88351
"""A0A024QZW4""","""ENSP00000498054""","""ENST00000648987""","""ENSG00000124786""",21601,1913978
"""A0A024QZW4""","""ENSP00000518269""","""ENST00000710437""","""ENSG00000124786""",21601,1913978
"""A0A024R3M2""","""ENSP00000489896""","""ENST00000638157""","""ENSG00000023171""",29214,1925037
"""A0A024R5G9""","""ENSP00000435822""","""ENST00000526010""","""ENSG00000174080""",2531,1861434
…,…,…,…,…,…
"""X6RGV9""","""ENSP00000403953""","""ENST00000420055""","""ENSG00000162639""",26400,1913965
"""X6RI37""","""ENSP00000408275""","""ENST00000451838""","""ENSG00000108061""",15454,1927197
"""X6RI37""","""ENSP00000510700""","""ENST00000689997""","""ENSG00000108061""",15454,1927197


In [15]:
mouse_convert_symbol = pl.read_csv(
    "../Data/Data_HNscore/mouse_convert_symbol.tsv", # Dowonload from Ensembl Biomart
    separator="\t"
).with_columns(
    pl.col("MGI ID").str.replace("MGI:", "").cast(pl.Int64).alias("mgi_gene_id")
).drop(
    "MGI ID"
).rename(
    {
        "Gene stable ID": "Mouse Gene ID",
        "MGI symbol": "GENEID"
    }
)

human_hit_to_mgiid_up = togoid_convert_to_mouse_up.join(
    mouse_convert_symbol,
    on="mgi_gene_id",
    how="left",
    coalesce=True
)

human_hit_to_mgiid_down = togoid_convert_to_mouse_down.join(
    mouse_convert_symbol,
    on="mgi_gene_id",
    how="left",
    coalesce=True
)

display(human_hit_to_mgiid_up.head())
display(human_hit_to_mgiid_down.head())

uniprot_id,ensembl_protein_id,ensembl_transcript_id,ensembl_gene_id,hgnc_id,mgi_gene_id,Mouse Gene ID,GENEID
str,str,str,str,i64,i64,str,str
"""A0A024QZW4""","""ENSP00000518269""","""ENST00000710437""","""ENSG00000124786""",21601,1913978,"""ENSMUSG00000021432""","""Slc35b3"""
"""A0A024QZW4""","""ENSP00000498054""","""ENST00000648987""","""ENSG00000124786""",21601,1913978,"""ENSMUSG00000021432""","""Slc35b3"""
"""A0A024R3B9""","""ENSP00000435411""","""ENST00000525823""","""ENSG00000109846""",2389,88516,"""ENSMUSG00000032060""","""Cryab"""
"""A0A024R3B9""","""ENSP00000435960""","""ENST00000528961""","""ENSG00000109846""",2389,88516,"""ENSMUSG00000032060""","""Cryab"""
"""A0A024R3B9""","""ENSP00000435046""","""ENST00000533280""","""ENSG00000109846""",2389,88516,"""ENSMUSG00000032060""","""Cryab"""


uniprot_id,ensembl_protein_id,ensembl_transcript_id,ensembl_gene_id,hgnc_id,mgi_gene_id,Mouse Gene ID,GENEID
str,str,str,str,i64,i64,str,str
"""A0A024QZP7""","""ENSP00000397973""","""ENST00000448257""","""ENSG00000170312""",1722,88351,"""ENSMUSG00000019942""","""Cdk1"""
"""A0A024QZW4""","""ENSP00000498054""","""ENST00000648987""","""ENSG00000124786""",21601,1913978,"""ENSMUSG00000021432""","""Slc35b3"""
"""A0A024QZW4""","""ENSP00000518269""","""ENST00000710437""","""ENSG00000124786""",21601,1913978,"""ENSMUSG00000021432""","""Slc35b3"""
"""A0A024R3M2""","""ENSP00000489896""","""ENST00000638157""","""ENSG00000023171""",29214,1925037,"""ENSMUSG00000040111""","""Gramd1b"""
"""A0A024R5G9""","""ENSP00000435822""","""ENST00000526010""","""ENSG00000174080""",2531,1861434,"""ENSMUSG00000083282""","""Ctsf"""


&nbsp;

&nbsp;


In [16]:
HNscore_sample_count_checked_up = human_hit_to_mgiid_up.join(
    HNscore_sample_validate,
    on="GENEID",
    how="left",
    coalesce=True
).filter(
    pl.col("HN5(all)").is_not_null()
).sort(
    by=[f'HN{threshold}(all)'],
    descending=True
)

HNscore_sample_count_checked_down = human_hit_to_mgiid_down.join(
    HNscore_sample_validate,
    on="GENEID",
    how="left",
    coalesce=True
).filter(
    pl.col("HN5(all)").is_not_null()
).sort(
    by=[f'HN{threshold}(all)'],
    descending=True
)

# save the result to csv
HNscore_sample_count_checked_up.write_csv(
    f'../Data/Data_HNscore/HNscore_sample_count_HN{threshold}_mouse_checked_up.tsv',
    separator='\t'
)

HNscore_sample_count_checked_down.write_csv(
    f'../Data/Data_HNscore/HNscore_sample_count_HN{threshold}_mouse_checked_down.tsv',
    separator='\t'
)
display(HNscore_sample_count_checked_up)
display(HNscore_sample_count_checked_down)

uniprot_id,ensembl_protein_id,ensembl_transcript_id,ensembl_gene_id,hgnc_id,mgi_gene_id,Mouse Gene ID,GENEID,HN5(all),positive_samples_count_mouse,negative_samples_count_mouse,total_samples,positive_contribution_mouse,negative_contribution_mouse
str,str,str,str,i64,i64,str,str,i32,i32,i32,i32,str,str
"""A0A0G2JIW1""","""ENSP00000387691""","""ENST00000417199""","""ENSG00000232804""",5233,99517,"""ENSMUSG00000090877""","""Hspa1b""",149,25,0,30,"""25 / 30 samples positively con…","""0 / 30 samples negatively cont…"
"""A0A0G2JIW1""","""ENSP00000402651""","""ENST00000458062""","""ENSG00000212866""",5233,99517,"""ENSMUSG00000090877""","""Hspa1b""",149,25,0,30,"""25 / 30 samples positively con…","""0 / 30 samples negatively cont…"
"""A0A0G2JIW1""","""ENSP00000442789""","""ENST00000545241""","""ENSG00000224501""",5233,99517,"""ENSMUSG00000090877""","""Hspa1b""",149,25,0,30,"""25 / 30 samples positively con…","""0 / 30 samples negatively cont…"
"""P0DMV9""","""ENSP00000364801""","""ENST00000375650""","""ENSG00000204388""",5233,99517,"""ENSMUSG00000090877""","""Hspa1b""",149,25,0,30,"""25 / 30 samples positively con…","""0 / 30 samples negatively cont…"
"""P0DMV9""","""ENSP00000375399""","""ENST00000391555""","""ENSG00000212866""",5233,99517,"""ENSMUSG00000090877""","""Hspa1b""",149,25,0,30,"""25 / 30 samples positively con…","""0 / 30 samples negatively cont…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Q6PIW4""","""ENSP00000410811""","""ENST00000419119""","""ENSG00000132436""",13286,1890648,"""ENSMUSG00000035455""","""Fignl1""",-20,0,7,30,"""0 / 30 samples positively cont…","""7 / 30 samples negatively cont…"
"""Q6PIW4""","""ENSP00000483543""","""ENST00000615084""","""ENSG00000132436""",13286,1890648,"""ENSMUSG00000035455""","""Fignl1""",-20,0,7,30,"""0 / 30 samples positively cont…","""7 / 30 samples negatively cont…"
"""Q6PIW4""","""ENSP00000483126""","""ENST00000617389""","""ENSG00000132436""",13286,1890648,"""ENSMUSG00000035455""","""Fignl1""",-20,0,7,30,"""0 / 30 samples positively cont…","""7 / 30 samples negatively cont…"


uniprot_id,ensembl_protein_id,ensembl_transcript_id,ensembl_gene_id,hgnc_id,mgi_gene_id,Mouse Gene ID,GENEID,HN5(all),positive_samples_count_mouse,negative_samples_count_mouse,total_samples,positive_contribution_mouse,negative_contribution_mouse
str,str,str,str,i64,i64,str,str,i32,i32,i32,i32,str,str
"""F5H694""","""ENSP00000437799""","""ENST00000541261""","""ENSG00000130649""",2631,88607,"""ENSMUSG00000025479""","""Cyp2e1""",21,12,3,30,"""12 / 30 samples positively con…","""3 / 30 samples negatively cont…"
"""H0Y7H4""","""ENSP00000412754""","""ENST00000421586""","""ENSG00000130649""",2631,88607,"""ENSMUSG00000025479""","""Cyp2e1""",21,12,3,30,"""12 / 30 samples positively con…","""3 / 30 samples negatively cont…"
"""P05181""","""ENSP00000440689""","""ENST00000463117""","""ENSG00000130649""",2631,88607,"""ENSMUSG00000025479""","""Cyp2e1""",21,12,3,30,"""12 / 30 samples positively con…","""3 / 30 samples negatively cont…"
"""P05181""","""ENSP00000252945""","""ENST00000252945""","""ENSG00000130649""",2631,88607,"""ENSMUSG00000025479""","""Cyp2e1""",21,12,3,30,"""12 / 30 samples positively con…","""3 / 30 samples negatively cont…"
"""J3KR25""","""ENSP00000415416""","""ENST00000422053""","""ENSG00000101255""",16228,1345675,"""ENSMUSG00000032715""","""Trib3""",20,10,2,30,"""10 / 30 samples positively con…","""2 / 30 samples negatively cont…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Q8TBE9""","""ENSP00000302441""","""ENST00000304788""","""ENSG00000170191""",16140,1914561,"""ENSMUSG00000053916""","""Nanp""",-19,1,5,30,"""1 / 30 samples positively cont…","""5 / 30 samples negatively cont…"
"""Q9BUA6""","""ENSP00000223167""","""ENST00000223167""","""ENSG00000106436""",29825,1891705,"""ENSMUSG00000005474""","""Myl10""",-19,1,10,30,"""1 / 30 samples positively cont…","""10 / 30 samples negatively con…"
"""P04908""","""ENSP00000483842""","""ENST00000615868""","""ENSG00000278463""",4734,2448293,"""ENSMUSG00000069301""","""H2ac11""",-20,4,7,30,"""4 / 30 samples positively cont…","""7 / 30 samples negatively cont…"
